In [1]:
#模块加载
import numpy as np
import pandas as pd

#确定路径
import os
os.chdir('C:/Users/Administrator/Desktop/项目/付费结构对比/少三安卓/')

#格式
pd.set_option('display.float_format',lambda x : '%.2f' % x)

In [7]:
#数据读取
data1 = pd.read_csv('交易明细2020-01-24 00_00_00 _ 2020-01-26 23_59_59.csv',engine = 'python')
data2 = pd.read_csv('交易明细2020-01-07 00_00_00 _ 2020-01-09 23_59_59.csv',engine = 'python')
match_table = pd.read_excel('渠道对接人0228.xlsx')
match_table['渠道类型'] = '公会'

#数据合并（将公会属性数据筛选出来）
data1_match = pd.merge(data1,match_table,left_on = '渠道ID',right_on = '渠道ID',how = 'left')
data2_match = pd.merge(data2,match_table,left_on = '渠道ID',right_on = '渠道ID',how = 'left')

data1_match = data1_match[data1_match['渠道类型'] == '公会']
data2_match = data2_match[data2_match['渠道类型'] == '公会']

In [8]:
#定义函数
#1）以果盘uin为分组依据，对整月数据进行求和，目的是去重，求出单个用户整月的总流水
#2）以0,500,1000..等作为区间（该区间可自定义），对用户流水进行分类
#3）求出上述区间的付费人数与付费金额

def f1(df):
    df_g = df.groupby('用户名(果盘UIN)').sum()
    gcut = pd.cut(df_g['金额'],[0,500,1000,3000,5000,10000,9999999],right = False)
    df_g['区间'] = gcut.values
    df_gg = df_g.groupby('区间').agg([len,np.sum])
    df_gg['数量占比'] = df_gg['金额']['len'] / df_gg['金额']['len'].sum()
    df_gg['金额占比'] = df_gg['金额']['sum'] / df_gg['金额']['sum'].sum()
    df_gg['数量占比'] = df_gg['数量占比'].apply(lambda x : '%.1f%%'%(x*100))
    df_gg['金额占比'] = df_gg['金额占比'].apply(lambda x : '%.1f%%'%(x*100))
    df_gg.index = [0,500,1000,3000,5000,10000]
    return df_gg[['金额','数量占比','金额占比']]

data1_9 = f1(data1_match)
data2_10 = f1(data2_match)

#将两个月的数据进行合并
data = pd.merge(data1_9,data2_10,left_index = True,right_index = True)
data

金额_x           数量占比_x 金额占比_x    金额_y           数量占比_y 金额占比_y
          len       sum                   len       sum              
0     2040.00 170304.00  83.6%  19.3% 1006.00  88949.00  77.2%  11.9%
500    191.00 148828.00   7.8%  16.9%   94.00  68630.00   7.2%   9.2%
1000   155.00 288629.00   6.3%  32.7%  125.00 222639.00   9.6%  29.8%
3000    28.00 105877.00   1.1%  12.0%   53.00 183497.00   4.1%  24.6%
5000    27.00 168873.00   1.1%  19.1%   24.00 169885.00   1.8%  22.8%
10000    0.00       nan   0.0%   nan%    1.00  13124.00   0.1%   1.8%